# Clustering Crypto

In [1]:
# Initial imports
!pip install -U altair
import requests
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from altair import Chart, X, Y, Axis, SortField
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

     |████████████████████████████████| 727 kB 8.3 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### Fetching Cryptocurrency Data

In [10]:
# Use the following endpoint to fetch json data
import json
from pprint import pprint

url = "https://min-api.cryptocompare.com/data/all/coinlist"
res = requests.get(url)
res
data = res.json()
json_data = json.dumps(data)

In [16]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
df = pd.DataFrame(data["Data"])
df = df.T
crypto_df = df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
433,433 Token,N/A,False,N/A,NaN,NaN


### Data Preprocessing

In [17]:
df.keys()

Index(['Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
       'CoinName', 'FullName', 'Description', 'AssetTokenStatus', 'Algorithm',
       'ProofType', 'SortOrder', 'Sponsored', 'Taxonomy', 'Rating',
       'IsTrading', 'TotalCoinsMined', 'BlockNumber', 'NetHashesPerSecond',
       'BlockReward', 'BlockTime', 'AssetLaunchDate', 'MaxSupply',
       'MktCapPenalty', 'PlatformType', 'BuiltOn', 'SmartContractAddress',
       'DecimalPoints', 'Difficulty', 'IsUsedInDefi'],
      dtype='object')

In [18]:
# Keep only necessary columns:
# Attention: TotalCoinSupply is not an index. I used MaxSupply instead.
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply'
crypto_df =crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
433,433 Token,N/A,False,N/A,NaN,NaN


In [19]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df["IsTrading"] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
611,SixEleven,SHA-256,True,PoW,0,0


In [21]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df["Algorithm"] != "N/A"]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
611,SixEleven,SHA-256,True,PoW,0,0
808,808,SHA-256,True,PoW/PoS,0,0


In [22]:
# Remove the "IsTrading" column
crypto_df.drop("IsTrading", axis=1, inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0


In [23]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna(axis=0, how="any")
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0


In [24]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] > 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
NVC,NovaCoin,Scrypt,PoW/PoS,3.08624e+06,-1
XCP,CounterParty,SHA-256,PoW,2.61521e+06,-1
NSR,NuShares,PoS,PoS,5.85024e+09,0
MONA,MonaCoin,Scrypt,PoW,8.00997e+07,-1
TRI,Triangles Coin,X13,PoW/PoS,166715,0


In [25]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df.iloc[:] != 'N/A'].dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
NVC,NovaCoin,Scrypt,PoW/PoS,3.08624e+06,-1
XCP,CounterParty,SHA-256,PoW,2.61521e+06,-1
NSR,NuShares,PoS,PoS,5.85024e+09,0
MONA,MonaCoin,Scrypt,PoW,8.00997e+07,-1
TRI,Triangles Coin,X13,PoW/PoS,166715,0


In [26]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = pd.DataFrame(crypto_df["CoinName"], index=crypto_df.index)
coins_name.head()

,CoinName
NVC,NovaCoin
XCP,CounterParty
NSR,NuShares
MONA,MonaCoin
TRI,Triangles Coin


In [27]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop("CoinName", axis=1)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
NVC,Scrypt,PoW/PoS,3.08624e+06,-1
XCP,SHA-256,PoW,2.61521e+06,-1
NSR,PoS,PoS,5.85024e+09,0
MONA,Scrypt,PoW,8.00997e+07,-1
TRI,X13,PoW/PoS,166715,0


In [28]:
# Create dummy variables for text features
X = pd.get_dummies(data=crypto_df, columns=["Algorithm", "ProofType"])
X.head()

,TotalCoinsMined,MaxSupply,Algorithm_BEP2 Token,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake2b,Algorithm_C31,Algorithm_CryptoNight,Algorithm_CryptoNight-Heavy,Algorithm_CryptoNight-Lite,...,ProofType_PoSA,ProofType_PoW,ProofType_PoW/PoS,ProofType_PoW/PoSe,ProofType_PoW/nPoS,ProofType_Proof of Authority,ProofType_SPoS,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
NVC,3.08624e+06,-1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
XCP,2.61521e+06,-1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
NSR,5.85024e+09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MONA,8.00997e+07,-1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
TRI,166715,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [29]:
# Standardize data
X = StandardScaler().fit_transform(X)
X[:5]

array([[-0.18138716, -0.16532847, -0.09853293, -0.14002801, -0.09853293,
        -0.14002801, -0.14002801, -0.14002801, -0.09853293, -0.09853293,
        -0.24743583, -0.09853293, -0.09853293, -0.09853293, -0.09853293,
        -0.26863534, -0.09853293, -0.1723455 , -0.09853293, -0.14002801,
        -0.09853293, -0.09853293, -0.09853293, -0.09853293, -0.09853293,
        -0.14002801, -0.09853293, -0.09853293, -0.09853293, -0.2       ,
        -0.09853293, -0.1723455 , -0.09853293, -0.36115756, -0.14002801,
        -0.09853293, -0.09853293,  2.11510856, -0.09853293, -0.09853293,
        -0.09853293, -0.09853293, -0.09853293, -0.2       , -0.09853293,
        -0.2       , -0.09853293, -0.24743583, -0.09853293, -0.09853293,
        -0.14002801, -0.09853293, -0.30779351, -0.09853293, -0.09853293,
        -1.        ,  1.9306146 , -0.09853293, -0.09853293, -0.09853293,
        -0.09853293, -0.09853293, -0.09853293, -0.09853293],
       [-0.18139057, -0.16532847, -0.09853293, -0.14002801, -0.

### Reducing Dimensions Using PCA

In [30]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
principal_components = pca.fit_transform(X)
principal_components

array([[-6.80294633e-01,  9.14893837e-01, -1.74404403e+00],
       [ 1.39335820e-01, -1.69412645e+00,  1.73876072e-01],
       [-4.38943777e-01,  1.41727641e+00,  8.48048077e-01],
       [-3.63820089e-01, -9.99810618e-01, -4.06498267e-01],
       [-7.52164526e-01,  1.70636246e+00, -2.37836835e+00],
       [-4.61734962e-01,  1.41827462e+00,  8.49636841e-01],
       [-1.76665748e-01,  2.20557301e-01, -1.16370266e+00],
       [-3.64129439e-01, -9.99797075e-01, -4.06476705e-01],
       [-5.84529784e-01,  9.65775241e-01, -6.86012339e-01],
       [-6.72480786e-01,  9.14595905e-01, -1.74456802e+00],
       [-6.03673861e-01,  1.93170411e+00,  4.64200916e+00],
       [-3.63193252e-01, -6.82818832e-01, -2.40275212e-01],
       [-3.03231448e-01, -1.37788786e+00,  1.18144152e-01],
       [-3.02609434e-01, -1.19084471e+00,  4.19424881e-01],
       [-6.18221549e-01,  5.36759866e-01, -1.21950122e+00],
       [ 1.47525276e-01, -1.69448498e+00,  1.73305255e-01],
       [-5.35505619e-01,  1.87954748e+00

In [31]:
# Create a DataFrame with the principal components data
n_components=3
col_names = [f"PC {i}" for i in range(1, n_components + 1)]
pca_df = pd.DataFrame(principal_components, columns=col_names, index=crypto_df.index)
pca_df.head()

,PC 1,PC 2,PC 3
NVC,-0.680295,0.914894,-1.744044
XCP,0.139336,-1.694126,0.173876
NSR,-0.438944,1.417276,0.848048
MONA,-0.363820,-0.999811,-0.406498
TRI,-0.752165,1.706362,-2.378368


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [32]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using altair scatter plot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
elbow_graph = alt.Chart(df_elbow).mark_line().encode(x='k', y='inertia').properties(title="Elbow Curve")

elbow_graph

alt.Chart(...)

Running K-Means with `k=<your best value for k here>`

In [33]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat([crypto_df, pca_df], axis=1, sort=False)
clustered_df["CoinName"] = coins_name["CoinName"]
clustered_df["Class"] = model.labels_
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,MaxSupply,PC 1,PC 2,PC 3,CoinName,Class
NVC,Scrypt,PoW/PoS,3.08624e+06,-1,-0.680295,0.914894,-1.744044,NovaCoin,3
XCP,SHA-256,PoW,2.61521e+06,-1,0.139336,-1.694126,0.173876,CounterParty,1
NSR,PoS,PoS,5.85024e+09,0,-0.438944,1.417276,0.848048,NuShares,0
MONA,Scrypt,PoW,8.00997e+07,-1,-0.363820,-0.999811,-0.406498,MonaCoin,1
TRI,X13,PoW/PoS,166715,0,-0.752165,1.706362,-2.378368,Triangles Coin,3


### Visualizing Results

#### 2D-Scatter with Clusters

In [37]:
# Create a 2D-Scatter with the PCA data and the clusters
pca_graph = alt.Chart(
    clustered_df).mark_point().encode(
    x="PC 1",
    y="PC 2",
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "MaxSupply", "Class"],
    color= alt.Color("Class",scale=alt.Scale(scheme='dark2'))
).properties(title="PC 1 vs PC 2 ").interactive()
    
pca_graph

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [38]:
# Table with tradable cryptos
display(clustered_df)

,Algorithm,ProofType,TotalCoinsMined,MaxSupply,PC 1,PC 2,PC 3,CoinName,Class
NVC,Scrypt,PoW/PoS,3.08624e+06,-1,-0.680295,0.914894,-1.744044,NovaCoin,3
XCP,SHA-256,PoW,2.61521e+06,-1,0.139336,-1.694126,0.173876,CounterParty,1
NSR,PoS,PoS,5.85024e+09,0,-0.438944,1.417276,0.848048,NuShares,0
MONA,Scrypt,PoW,8.00997e+07,-1,-0.363820,-0.999811,-0.406498,MonaCoin,1
TRI,X13,PoW/PoS,166715,0,-0.752165,1.706362,-2.378368,Triangles Coin,3
...,...,...,...,...,...,...,...,...,...
BTS,SHA-512,PoS,2.99482e+09,3600570502,-0.456021,1.844574,0.972851,Bitshares,0
ETH,Ethash,PoW,1.13994e+08,-1,-0.253758,-1.680506,0.328124,Ethereum,1
WAVES,Leased POS,LPoS,103934554,-1,-0.602312,1.879249,3.306293,Waves,0
ADA,Ouroboros,PoS,3.18092e+10,45000000000,-0.007300,1.800534,0.846680,Cardano,0


In [39]:
# Print the total number of tradable cryptocurrencies
print(f"There are {clustered_df.shape[0]} tradable cryptocurrencies.")

There are 104 tradable cryptocurrencies.


#### Scatter Plot with Tradable Cryptocurrencies

In [41]:
# Scale data to create the scatter plot
mm_scaler = MinMaxScaler()
plot_data = mm_scaler.fit_transform(
    clustered_df[["MaxSupply", "TotalCoinsMined"]]
)
plot_df = pd.DataFrame(
    plot_data, columns=["MaxSupply", "TotalCoinsMined"], index=clustered_df.index
)
plot_df["CoinName"] = clustered_df["CoinName"]
plot_df["Class"] = clustered_df["Class"]
plot_df.head()

,MaxSupply,TotalCoinsMined,CoinName,Class
NVC,0.000000e+00,0.000003,NovaCoin,3
XCP,0.000000e+00,0.000002,CounterParty,1
NSR,9.999833e-13,0.005850,NuShares,0
MONA,0.000000e+00,0.000080,MonaCoin,1
TRI,9.999833e-13,0.000000,Triangles Coin,3


In [43]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
fig = alt.Chart(clustered_df).mark_point().encode(
    x="TotalCoinsMined",
    y="MaxSupply",
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "MaxSupply", "Class"],
    color=alt.Color("Class",scale=alt.Scale(scheme='dark2'))
).properties(title="Total Coin Supply vs. Total Coins Mined (10,000s)").interactive()
    
fig

alt.Chart(...)